Gdrive

In [1]:
gdrive_project_pathname = '/My Drive/w210'  #@param {type: "string"}

In [2]:
import os.path
from google.colab import drive

# Mount the current user's Google Drive
GOOGLE_DRIVE_MOUNT_POINT = "/content/drive"
print("Mounting Google Drive beneath %s" % GOOGLE_DRIVE_MOUNT_POINT)
drive.mount(GOOGLE_DRIVE_MOUNT_POINT, force_remount = True)

# Build the pathname to the project's folder residing beneath the current user's Google Drive
if not gdrive_project_pathname.startswith("/"):
  gdrive_project_pathname = "/" + gdrive_project_pathname
abs_project_pathname = GOOGLE_DRIVE_MOUNT_POINT + gdrive_project_pathname
print("Project folder: %s" % gdrive_project_pathname)
data_project_pathname = abs_project_pathname + '/data/'

Mounting Google Drive beneath /content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Project folder: /My Drive/w210


In [2]:
# !pip3 install -q --upgrade ipython==5.5.0
# !pip3 install -q --upgrade ipykernel==4.6.0

     |████████████████████████████████| 768kB 3.4MB/s 
     |████████████████████████████████| 245kB 16.7MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.0 which is incompatible.
     |████████████████████████████████| 112kB 3.5MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 4.6.0 which is incompatible.


GEE

In [3]:
!pip3 install earthengine-api 

In [4]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-

In [5]:
import ee
import geopandas
import numpy as np
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from shapely.geometry import box
import warnings
from functools import reduce
from math import sin, cos, sqrt, atan2, radians

In [6]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=P_NVJYsDpbFDqbVwxLrEmqA3jrmxRVd6bCL2bVCGI_Q&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1QEW5A-7zDte-tgc6SzUHCx9VFvsLfhXj-0_JqBnRo49kk9_QZx2Xd8

Successfully saved authorization token.


Create a dictionary of countries with its boundary coordinates

In [9]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
country_polygon_dict = dict()
for index, row in world.iterrows():
    if 'MultiPolygon' in str(type(row['geometry'])):
        i = 0
        for poly in row['geometry']:
            # Remove small area that's not needed for sampling
            if poly.area > 5:
                # Area is in the unit of squre degrees
                country_polygon_dict[row['iso_a3']+'_'+str(i)] = (poly, round(poly.area*0.2))
                i +=1 
    # Here implies a Polygon
    elif row['geometry'].area > 5:
        country_polygon_dict[row['iso_a3']] = (row['geometry'], round(row['geometry'].area*0.2))

In [ ]:
world[world.iso_a3 == 'ARG'].head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
9,44293293,South America,Argentina,ARG,879400.0,"MULTIPOLYGON (((-68.63401 -52.63637, -68.25000..."


In [ ]:
# Remove miscellaneous countries
del country_polygon_dict['-99']
del country_polygon_dict['-99_0']
del country_polygon_dict['-99_1']
del country_polygon_dict['-99_2']

In [ ]:
country_polygon_dict

In [10]:
def country_bounds(area_of_interest_shapely, country_code, edge_len):

    return_list_coords = []
    min_lon, min_lat, max_lon, max_lat = area_of_interest_shapely.bounds
    dict_coords = {}
    point_list = []
    lat_list = []
    lon_list = []
    counter = 0
    center_lon = min_lon
    center_lat = max_lat
    # print(center_lat,min_lat,  center_lon, max_lon)
    while center_lon < (max_lon):
        while center_lat > (min_lat):
            # print(center_lat, center_lon)
            point_list.append(counter)
            lat_list.append(center_lat)
            lon_list.append(center_lon)
            counter +=1
            center_lat -= edge_len
        
        center_lon += edge_len
        center_lat = max_lat
        
    dict_coords['POINTS'] = point_list
    dict_coords['Latitude'] = lat_list
    dict_coords['Longitude'] = lon_list
    # print("Total coords: ", counter)
    df = pd.DataFrame(dict_coords)
    gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))
    gdf.crs = "EPSG:4326"
    point_within_country = geopandas.sjoin(gdf, world[world.iso_a3 == country_code], how="inner", op='intersects')
    for index, row in point_within_country.iterrows():
      (lat, lon) = (row['Latitude'], row['Longitude'])
      # (center_lat, center_lon) = (point_within_arg['Latitude'],	point_within_arg['Longitude'])
      return_list_coords.append((lat, lon))
    
    return return_list_coords

In [15]:
test_arg = country_bounds(country_polygon_dict['ARG_0'][0], 'ARG', 0.5)

In [ ]:
len(test_arg)

list

In [ ]:
# https://gis.stackexchange.com/questions/6412/generate-points-that-lie-inside-polygon
    
    
# https://stackoverflow.com/questions/58802921/using-geopandas-how-to-randomly-select-in-each-polygon-5-points-by-sampling-met
    
    
# https://developers.google.com/earth-engine/geometries

Pull Sentinel-2 Data

In [ ]:

# def satellite_imagery_Sentinel(area_of_interest_shapely, country_code,
#                       start_date, end_date, 
#                       plot_option, resolution=30):
    
#     area_of_interest_ee = ee.Geometry.Polygon(list(area_of_interest_shapely.exterior.coords))
    
#     # Create image collection that contains the area of interest
#     img_collect = (ee.ImageCollection('COPERNICUS/S2')
#                  .filterDate(start_date, end_date)
#                  .filterBounds(area_of_interest_ee)
#                     # Remove image that's too small (likely to be partial image)
#                     # Size of a full image: 1,276,131,371; size of a partial image: 276,598,191
#                  .filter(ee.Filter.gt('system:asset_size', 800000000))
#                  .filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",50))
    
# #         img_collect = img_collect.filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",20) 
    
#     assert (img_collect.size().getInfo()>0), "No valid image"
#     print("Total number of images in the collection: ", img_collect.size().getInfo())
        
#     # Extract tile information from each image
#     # Note: tiles can overlap a little bit
#     unique_tiles = set([item['properties']['MGRS_TILE'] for item in img_collect.getInfo()['features']])
#     if len(unique_tiles) > 1:
#         warnings.warn('Multiple tiles are selected. Proceed with caution.')
#         print('Number of tiles selected: ', len(unique_tiles))
# #     if img_collect_no_partial.size().getInfo() < img_collect.size().getInfo():
# #         warnings.warn('There are partial images in the collection. Proceed with caution.')
# #         print('Number of partial images: ', img_collect.size().getInfo()-img_collect_no_partial.size().getInfo())
        
#     # Reference: https://www.satimagingcorp.com/satellite-sensors/other-satellite-sensors/sentinel-2a/
#     band_blue = 'B2' #10m
#     band_green = 'B3' #10m
#     band_red = "B4"  #10m
#     band_nir = 'B8'  #10m
    
#     def calc_NDVI(img):
#         ndvi = ee.Image(img.normalizedDifference([band_nir, band_red])).rename(["ndvi"]).copyProperties(img, img.propertyNames())
#         composite = img.addBands(ndvi)
#         return composite
    
#     # SAVI = ((NIR – Red) / (NIR + Red + L)) x (1 + L)
#     def calc_SAVI(img):
#         """A function to compute Soil Adjusted Vegetation Index."""
#         savi =  ee.Image(img.expression(
#             '(1 + L) * float(nir - red)/ (nir + red + L)',
#             {
#                 'nir': img.select(band_nir),
#                 'red': img.select(band_red),
#                 'L': 0.5
#             })).rename(["savi"]).copyProperties(img, img.propertyNames())
#         composite = img.addBands(savi)
#         return composite

#     # EVI = 2.5 * ((NIR – Red) / ((NIR) + (C1 * Red) – (C2 * Blue) + L))
#     #     C1=6, C2=7.5, and L=1
#     def calc_EVI(img):
#         """A function to compute Soil Adjusted Vegetation Index."""
#         evi = ee.Image(img.expression(
#           '(2.5) * float(nir - red)/ ((nir) + (C1*red) - (C2*blue) + L)',
#           {   
#               'nir': img.select(band_nir),
#               'red': img.select(band_red),
#               'blue': img.select(band_blue),
#               'L': 0.2,
#               'C1': 6,
#               'C2': 7.5
#           })).rename(["evi"]).copyProperties(img, img.propertyNames())
#         composite = img.addBands(evi)
#         return composite
                   
#     def add_landcover(img):
#         landcover = ee.Image("USGS/GFSAD1000_V1")
#         composite = img.addBands(landcover)
#         return composite
    
#     def calc_YYYYMM(img):
#         return img.set('YYYYMM', img.date().format("YYYYMM"))
    
#     def add_ee_layer(self, ee_object, vis_params, name):
#         try:    
#             if isinstance(ee_object, ee.image.Image):    
#                 map_id_dict = ee.Image(ee_object).getMapId(vis_params)
#                 folium.raster_layers.TileLayer(
#                     tiles = map_id_dict['tile_fetcher'].url_format,
#                     attr = 'Google Earth Engine',
#                     name = name,
#                     overlay = True,
#                     control = True
#                     ).add_to(self)
#             elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
#                 ee_object_new = ee_object.median()
#                 map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
#                 folium.raster_layers.TileLayer(
#                     tiles = map_id_dict['tile_fetcher'].url_format,
#                     attr = 'Google Earth Engine',
#                     name = name,
#                     overlay = True,
#                     control = True
#                     ).add_to(self)
#             elif isinstance(ee_object, ee.geometry.Geometry):    
#                 folium.GeoJson(
#                         data = ee_object.getInfo(),
#                         name = name,
#                         overlay = True,
#                         control = True
#                     ).add_to(self)
#             elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
#                 ee_object_new = ee.Image().paint(ee_object, 0, 2)
#                 map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
#                 folium.raster_layers.TileLayer(
#                         tiles = map_id_dict['tile_fetcher'].url_format,
#                         attr = 'Google Earth Engine',
#                         name = name,
#                         overlay = True,
#                         control = True
#                     ).add_to(self)

#         except:
#             print("Could not display {}".format(name))

#     # Add EE drawing method to folium.
#     folium.Map.add_ee_layer = add_ee_layer
    
#     img_collect_calc = img_collect.map(calc_YYYYMM).map(calc_NDVI).map(calc_SAVI).map(calc_EVI).map(add_landcover)
    
#     unique_month = list(set([item['properties']['YYYYMM'] for item in img_collect_calc.getInfo()['features']]))
#     unique_month.sort()
#     print(unique_month)
    
#     if len(unique_month) > 0:
#         warnings.warn('There are null values in the output DataFrame. Proceed with caution.')
    

#     # min_lon, min_lat, max_lon, max_lat = area_of_interest_shapely.bounds
#     # rectangle_top_right = [max_lat, max_lon]
#     # rectangle_top_left = [max_lat, min_lon]
#     # rectangle_bottom_left = [min_lat, min_lon]
#     # rectangle_bottom_right = [min_lat, max_lon]
#     list_coords = []
#     # dict_coords = {}
#     # point_list = []
#     # lat_list = []
#     # lon_list = []
#     # area_rect = (max_lat - min_lat) * (max_lon - min_lon)
#     # print("area_rect: ", area_rect)
#     # edge_len = 0.1
#     #area_of_interest_ee = ee.Geometry.Rectangle([center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2])
#     #start at rectangle_top_left - go 0.1 while center_lon <max_lon
#     # for lonitude in range(min_lon, max_lon, edge_len):
#     #     for latitude in range(max_lat, min_lat, -edge_len):
#     #         print(latitude, longitude)  
#     # center_lon = min_lon
#     # center_lat = max_lat
#     # # print(center_lat,min_lat,  center_lon, max_lon)
#     img_calc_month_dict = dict()
#     # temp_dict = dict()
#     # while center_lon < max_lon:
#     #     while center_lat > min_lat:
#     #         # print(center_lat, center_lon)
#     #         # print(center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2)
#     #         rect_of_interest_ee = ee.Geometry.Rectangle([center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2])
      
#     #         center_lat -= edge_len
#     #         # center_lon += edge_len

#     #             # Create image collection that contains the area of interest
#     #         rect_img_collect = (ee.ImageCollection('COPERNICUS/S2')
#     #                           .filterDate(start_date, end_date)
#     #                           .filterBounds(rect_of_interest_ee)
#     #                               # Remove image that's too small (likely to be partial image)
#     #                               # Size of a full image: 1,276,131,371; size of a partial image: 276,598,191
#     #                           # .filter(ee.Filter.gt('system:asset_size', 800000000))
#     #                           # .filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",50)
#     #                           )

#     #         if rect_img_collect:
#     #             rect_img_collect_calc = rect_img_collect.map(calc_YYYYMM).map(calc_NDVI).map(calc_SAVI).map(calc_EVI)
#     #             rect_unique_month = list(set([item['properties']['YYYYMM'] for item in rect_img_collect_calc.getInfo()['features']]))
#     #             rect_unique_month.sort()
#     #             # img_calc_month_dict = dict()
#     #             # temp_dict = dict()
#     #             list_coords.append((center_lat, center_lon))
#     #             point_list.append(counter)
#     #             lat_list.append(center_lat)
#     #             lon_list.append(center_lon)
#     #             dict_loc = str(center_lat) + ', ' + str(center_lon)
#     #             # for month in rect_unique_month:
#     #             #     img_calc_month_dict[dict_loc + month] = rect_img_collect_calc.filter(ee.Filter.eq('YYYYMM',month)).median()
#     #             #     img_calc_month2 = img_calc_month_dict[dict_loc + month].addBands(ee.Image.pixelLonLat())
#     #             #     # EEException: Output of image computation is too large (20 bands for 851968 pixels = 126.8 MiB > 80.0 MiB).
#     #             #     #     If this is a reduction, try specifying a larger 'tileScale' parameter.
#     #             #     # EEException: ReduceRegion.AggregationContainer: Valid tileScales are 1 to 16.
#     #             #     data_month_lst = img_calc_month2.reduceRegion(reducer=ee.Reducer.toList(), \
#     #             #                                                         geometry=rect_of_interest_ee, maxPixels=1e13, scale=resolution)

#     #             #     lat_series = pd.Series(np.array((ee.Array(data_month_lst.get("latitude")).getInfo())), name="lat")
#     #             #     lon_series = pd.Series(np.array((ee.Array(data_month_lst.get("longitude")).getInfo())), name="lon")
#     #             #     ndvi_series = pd.Series(np.array((ee.Array(data_month_lst.get("ndvi")).getInfo())), name=month+'_NDVI')
#     #             #     savi_series = pd.Series(np.array((ee.Array(data_month_lst.get("savi")).getInfo())), name=month+'_SAVI')
#     #             #     evi_series = pd.Series(np.array((ee.Array(data_month_lst.get("evi")).getInfo())), name=month+'_EVI')
#     #             #     temp_dict[dict_loc + month] = pd.concat([lat_series, lon_series, ndvi_series, savi_series, evi_series], axis=1)
      
#     #         # print("completed:" ,center_lat, center_lon)
#     #         ###end of while loop for lat
#     #     center_lon += edge_len
#     #     center_lat = max_lat  
#     # df_lst = list(temp_dict.values())
#     # # print("df_lst: ", df_lst)
#     # if df_lst:
#     #     out_df = reduce(lambda left, right: pd.merge(left,right,on=['lat', 'lon']), df_lst)

#     #     out_df.to_csv(abs_project_pathname + '/time_series_ID_scale10_SENTINEL2_1C_06202020.csv')
# #     # Output the column names that have null values
# #     if len(out_df.columns[out_df.isnull().any()]) > 0:
# #         warnings.warn('There are null values in the output DataFrame. Proceed with caution.')

#     # Create a folium map object.
#     center_lon, center_lat = list(area_of_interest_shapely.centroid.coords)[0]
#     print("center_lon, center_lat : ", center_lon, center_lat)
#     myMap = folium.Map(location=[center_lat, center_lon], zoom_start=5)
#     # Add the box around the area of interest
#     folium.GeoJson(area_of_interest_shapely, name="Area of Interest").add_to(myMap)
#     # folium.Marker(location=rectangle_top_right).add_to(myMap)
#     # folium.Marker(location=[max_lat, min_lon]).add_to(myMap)
#     # folium.Marker(location=[min_lat, min_lon]).add_to(myMap)
#     # folium.Marker(location=[min_lat, max_lon]).add_to(myMap)
#     # list_coords = country_bounds(area_of_interest_shapely, country_code, 0.1)
#     # print("total sub rects: ", len(list_coords))
#     # for coord in list_coords:
#     #     center_lat, center_lon = coord[0], coord[1]
#     #     rect_of_interest_shapely = box(center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2)
#     #     # Add the box around the area of interest
#     #     folium.GeoJson(rect_of_interest_shapely).add_to(myMap)

#     visParams = {'min':0, 'max':1, 'palette': ['red', 'yellow', 'green']}
#     if plot_option == 'NDVI':
#         for month in unique_month:
#             myMap.add_ee_layer(img_calc_month_dict[month].select("ndvi"), visParams, name=plot_option+' '+month)
#     elif plot_option == 'SAVI':
#         for month in unique_month:
#             myMap.add_ee_layer(img_calc_month_dict[month].select("savi"), visParams, name=plot_option+' '+month)
#     elif plot_option == 'EVI':
#         for month in unique_month:
#             myMap.add_ee_layer(img_calc_month_dict[month].select("evi"), visParams, name=plot_option+' '+month)

#     # Add a layer control panel to the map.
#     myMap.add_child(folium.LayerControl())

# #     return myMap, out_df
#     return myMap

In [21]:
#######old code - changes

# def satellite_imagery_Sentinel(area_of_interest_shapely, country_code,
#                       start_date, end_date, 
#                       plot_option, resolution=30):
def satellite_imagery_Sentinel(area_of_interest_shapely, coord_list, edge_len,
                      start_date, end_date, 
                      plot_option, resolution=30):
    
    area_of_interest_ee = ee.Geometry.Polygon(list(area_of_interest_shapely.exterior.coords))

    
    # Create image collection that contains the area of interest
    img_collect = (ee.ImageCollection('COPERNICUS/S2')
                 .filterDate(start_date, end_date)
                 .filterBounds(area_of_interest_ee)
                    # Remove image that's too small (likely to be partial image)
                    # Size of a full image: 1,276,131,371; size of a partial image: 276,598,191
                 .filter(ee.Filter.gt('system:asset_size', 800000000))
                 .filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",50))
    
    assert (img_collect.size().getInfo()>0), "No valid image"
    print("Total number of images in the collection: ", img_collect.size().getInfo())
        
    # Extract tile information from each image
    # Note: tiles can overlap a little bit
    unique_tiles = set([item['properties']['MGRS_TILE'] for item in img_collect.getInfo()['features']])
    if len(unique_tiles) > 1:
        warnings.warn('Multiple tiles are selected. Proceed with caution.')
        print('Number of tiles selected: ', len(unique_tiles))
        
    # Reference: https://www.satimagingcorp.com/satellite-sensors/other-satellite-sensors/sentinel-2a/
    band_blue = 'B2' #10m
    band_green = 'B3' #10m
    band_red = "B4"  #10m
    band_nir = 'B8'  #10m
    
    def calc_NDVI(img):
        ndvi = ee.Image(img.normalizedDifference([band_nir, band_red])).rename(["ndvi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(ndvi)
        return composite
    
    # SAVI = ((NIR – Red) / (NIR + Red + L)) x (1 + L)
    def calc_SAVI(img):
        """A function to compute Soil Adjusted Vegetation Index."""
        savi =  ee.Image(img.expression(
            '(1 + L) * float(nir - red)/ (nir + red + L)',
            {
                'nir': img.select(band_nir),
                'red': img.select(band_red),
                'L': 0.5
            })).rename(["savi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(savi)
        return composite

    # EVI = 2.5 * ((NIR – Red) / ((NIR) + (C1 * Red) – (C2 * Blue) + L))
    #     C1=6, C2=7.5, and L=1
    def calc_EVI(img):
        """A function to compute Soil Adjusted Vegetation Index."""
        evi = ee.Image(img.expression(
          '(2.5) * float(nir - red)/ ((nir) + (C1*red) - (C2*blue) + L)',
          {   
              'nir': img.select(band_nir),
              'red': img.select(band_red),
              'blue': img.select(band_blue),
              'L': 0.2,
              'C1': 6,
              'C2': 7.5
          })).rename(["evi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(evi)
        return composite
                   
    def add_landcover(img):
        landcover = ee.Image("USGS/GFSAD1000_V1")
        composite = img.addBands(landcover)
        return composite
    
    def calc_YYYYMM(img):
        return img.set('YYYYMM', img.date().format("YYYYMM"))
    
    def add_ee_layer(self, ee_object, vis_params, name):
        try:    
            if isinstance(ee_object, ee.image.Image):    
                map_id_dict = ee.Image(ee_object).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
                ee_object_new = ee_object.median()
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.geometry.Geometry):    
                folium.GeoJson(
                        data = ee_object.getInfo(),
                        name = name,
                        overlay = True,
                        control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
                ee_object_new = ee.Image().paint(ee_object, 0, 2)
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                        tiles = map_id_dict['tile_fetcher'].url_format,
                        attr = 'Google Earth Engine',
                        name = name,
                        overlay = True,
                        control = True
                    ).add_to(self)

        except:
            print("Could not display {}".format(name))

    # Add EE drawing method to folium.
    folium.Map.add_ee_layer = add_ee_layer
    
    img_collect_calc = img_collect.map(calc_YYYYMM).map(calc_NDVI).map(calc_SAVI).map(calc_EVI).map(add_landcover)
    
    unique_month = list(set([item['properties']['YYYYMM'] for item in img_collect_calc.getInfo()['features']]))
    unique_month.sort()
    print(unique_month)
    
    if len(unique_month) > 0:
        warnings.warn('There are null values in the output DataFrame. Proceed with caution.')
    
    
    img_calc_month_dict = dict()
    temp_dict = dict()
    for month in unique_month:
        img_calc_month_dict[month] = img_collect_calc.filter(ee.Filter.eq('YYYYMM',month))

    # Create a folium map object.
    center_lon, center_lat = list(area_of_interest_shapely.centroid.coords)[0]
    print("center coords: ", center_lon, center_lat)
    myMap = folium.Map(location=[center_lat, center_lon], zoom_start=5)
    # Add the box around the area of interest
    folium.GeoJson(area_of_interest_shapely, name="Area of Interest").add_to(myMap)

    # adding box around the bounds of Argentina
    min_lon, min_lat, max_lon, max_lat = area_of_interest_shapely.bounds
    total_area_of_interest_shapely = box(min_lon, min_lat, max_lon, max_lat)
    folium.GeoJson(total_area_of_interest_shapely).add_to(myMap)

    ###adding boxes
    edge_len = edge_len #0.5
    list_coords = coord_list #country_bounds(area_of_interest_shapely, country_code, edge_len)
    print("total sub rects: ", len(list_coords))
    final_df = pd.DataFrame()
    for coord in list_coords:
        center_lat, center_lon = coord[0], coord[1]
        folium.Marker(location=[center_lat, center_lon]).add_to(myMap)
        rect_of_interest_shapely = box(center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2)
        # Add the box around the area of interest
        folium.GeoJson(rect_of_interest_shapely).add_to(myMap)
    
        # Specify area of interest (one for ee.Geometry, one for shapely)
        rect_of_interest_ee = ee.Geometry.Rectangle([center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2])
        
        # Create image collection that contains the area of interest
        rect_img_collect = (ee.ImageCollection('COPERNICUS/S2')
                        .filterDate(start_date, end_date)
                        .filterBounds(rect_of_interest_ee)
                        .filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",50))
        
        rect_img_collect_calc = rect_img_collect.map(calc_YYYYMM).map(calc_NDVI).map(calc_SAVI).map(calc_EVI)
            
        unique_month = list(set([item['properties']['YYYYMM'] for item in rect_img_collect_calc.getInfo()['features']]))
        unique_month.sort()
        
        if len(unique_month) > 0:
            warnings.warn('There are null values in the output DataFrame. Proceed with caution.')
            
        rect_img_calc_month_dict = dict()
        rect_temp_dict = dict()
        for month in unique_month:
            rect_img_calc_month_dict[month] = rect_img_collect_calc.filter(ee.Filter.eq('YYYYMM',month)).median()
            rect_img_calc_month2 = rect_img_calc_month_dict[month].addBands(ee.Image.pixelLonLat())
            data_month_lst = rect_img_calc_month2.reduceRegion(reducer=ee.Reducer.toList(), \
                                                               geometry=rect_of_interest_ee, maxPixels=1e13, scale=30, tileScale=16)
            lat_series = pd.Series(np.array((ee.Array(data_month_lst.get("latitude")).getInfo())), name="lat")
            lon_series = pd.Series(np.array((ee.Array(data_month_lst.get("longitude")).getInfo())), name="lon")
            ndvi_series = pd.Series(np.array((ee.Array(data_month_lst.get("ndvi")).getInfo())), name=month+'_NDVI')
            savi_series = pd.Series(np.array((ee.Array(data_month_lst.get("savi")).getInfo())), name=month+'_SAVI')
            evi_series = pd.Series(np.array((ee.Array(data_month_lst.get("evi")).getInfo())), name=month+'_EVI')
            temp_dict[month] = pd.concat([lat_series, lon_series, ndvi_series, savi_series, evi_series], axis=1)
        
        df_lst = list(temp_dict.values())
        out_df = reduce(lambda left, right: pd.merge(left,right,on=['lat', 'lon']), df_lst)
        final_df.append(out_df)
        print("Completed coord: ", center_lat, center_lon)

    final_df.to_csv(abs_project_pathname + '/time_series_ID_scale10_SENTINEL2_1C_06202020.csv')

    # original code segment - don't touch
    visParams = {'min':0, 'max':1, 'palette': ['red', 'yellow', 'green']}
    if plot_option == 'NDVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("ndvi"), visParams, name=plot_option+' '+month)
    elif plot_option == 'SAVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("savi"), visParams, name=plot_option+' '+month)
    elif plot_option == 'EVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("evi"), visParams, name=plot_option+' '+month)

    # Add a layer control panel to the map.
    myMap.add_child(folium.LayerControl())

#     return myMap, out_df
    return myMap

In [14]:
from shapely import wkt
from ast import literal_eval

In [15]:
world_df = pd.read_csv(data_project_pathname + '/Global_samples_Balanced_10.csv', index_col=[0])
world_df.head()
world_df['geometry'] = world_df['geometry'].apply(wkt.loads)
world_df['samples'] = world_df['samples'].apply(literal_eval)
world_df['boundaries'] = world_df['boundaries'].apply(literal_eval)
world_df['coords'] = world_df['coords'].apply(literal_eval)
world_gdf = geopandas.GeoDataFrame(world_df, geometry='geometry')
world_gdf.crs = "EPSG:3857"
world_gdf.head()

,country_code,geometry,area,samples,boundaries,coords
0,FJI,"POLYGON ((180.00000 -16.06713, 180.00000 -16.5...",0.588778,[],"[178.59683859511713, -17.01204167436804, 180.0...","[(-16.667132663642455, 178.69683859511713), (-..."
1,FJI,"POLYGON ((178.12557 -17.50481, 178.37360 -17.3...",0.983740,[],"[177.28504, -18.28799, 178.71806, -17.33992]","[(-17.639920000000004, 177.38504), (-17.739920..."
2,FJI,"POLYGON ((-179.79332 -16.02088, -179.91737 -16...",0.066993,[],"[-180.0, -16.555216566639196, -179.79332010904...","[(-16.120882256741226, -180.0), (-16.220882256..."
3,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",76.301964,"[[30.938887681661946, -1.3967093153271337], [3...","[29.339997592900346, -11.720938002166735, 40.3...","[(-4.550000000000001, 29.439997592900347), (-4..."
4,ESH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",8.603984,[],"[-17.06342322434257, 20.999752102130827, -8.66...","[(21.356425889592266, -16.96342322434257), (21..."


In [ ]:
# satellite_imagery_Sentinel(area_of_interest_shapely=country_polygon_dict['ARG_0'][0], country_code='ARG',
#                   start_date='2018-1-01', end_date='2018-2-28', plot_option='NDVI')

satellite_imagery_Sentinel(area_of_interest_shapely=world_gdf.iloc[85].geometry, coord_list= world_gdf.iloc[85].coords, edge_len = 0.1,
                  start_date='2018-1-01', end_date='2018-2-28', plot_option='NDVI')

Total number of images in the collection:  1447


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Multiple tiles are selected. Proceed with caution.


Number of tiles selected:  174
['201801', '201802']
center coords:  -65.45374942416133 -41.34592157885223
total sub rects:  19248


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:133: UserWarning: There are null values in the output DataFrame. Proceed with caution.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:180: UserWarning: There are null values in the output DataFrame. Proceed with caution.


Completed coord:  -49.31629485445453 -73.3154357571201


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:180: UserWarning: There are null values in the output DataFrame. Proceed with caution.


Completed coord:  -41.51629485445442 -71.6154357571202


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:180: UserWarning: There are null values in the output DataFrame. Proceed with caution.


Completed coord:  -41.616294854454424 -71.6154357571202


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:180: UserWarning: There are null values in the output DataFrame. Proceed with caution.


In [ ]:
min_lon, min_lat, max_lon, max_lat = country_polygon_dict['ARG_0'][0].bounds

NameError: ignored

In [ ]:
type(country_polygon_dict['ARG_0'][0])

shapely.geometry.polygon.Polygon

In [ ]:
min_lon, min_lat, max_lon, max_lat

(-73.4154357571201,
 -52.34998340612768,
 -53.628348965048744,
 -21.83231047942072)

starts from -21.83231047942072, -73.4154357571201
minus from lat goes down, end adds to lon goes across

In [ ]:
# for lonitude in range(min_lon, max_lon, edge_len):
#     for latitude in range(max_lat, min_lat, -edge_len):
#         print(latitude, longitude)   

# (-73.4154357571201,
#  -52.34998340612768,
#  -53.628348965048744,
#  -21.83231047942072)
dict_coords = {}
point_list = []
lat_list = []
lon_list = []
counter = 0
edge_len = 0.1
center_lon = min_lon
center_lat = max_lat
print(center_lat,min_lat,  center_lon, max_lon)
while center_lon < max_lon:
    while center_lat > min_lat:
        print(center_lat, center_lon)
        point_list.append(counter)
        lat_list.append(center_lat)
        lon_list.append(center_lon)
        # print(center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2)
        # if (center_lon-edge_len/2 in country_polygon_dict['ARG_0'][0]):
        #     print("within poly")
        #     counter +=1
        # if (center_lat-edge_len/2 in country_polygon_dict['ARG_0'][0]):
        #     print("within poly")
        #     counter +=1
        # if (center_lon+edge_len/2 in country_polygon_dict['ARG_0'][0]):
        #     print("within poly")
        #     counter +=1
        # if (center_lat+edge_len/2 in country_polygon_dict['ARG_0'][0]):
        #     print("within poly")
        #     counter +=1
        # else:
        #     print("outside poly")
        counter +=1
        center_lat -= edge_len
        # center_lon += edge_len
        
    
    center_lon += edge_len
    center_lat = max_lat
    # print("modified center_lon: ", center_lon, center_lat)
dict_coords['POINTS'] = point_list
dict_coords['Latitude'] = lat_list
dict_coords['Longitude'] = lon_list
print("Total coords: ", counter)

Streaming output truncated to the last 5000 lines.
-42.23231047942101 -55.315435757120504
-42.33231047942101 -55.315435757120504
-42.43231047942101 -55.315435757120504
-42.532310479421014 -55.315435757120504
-42.632310479421015 -55.315435757120504
-42.73231047942102 -55.315435757120504
-42.83231047942102 -55.315435757120504
-42.93231047942102 -55.315435757120504
-43.03231047942102 -55.315435757120504
-43.13231047942102 -55.315435757120504
-43.232310479421024 -55.315435757120504
-43.332310479421025 -55.315435757120504
-43.43231047942103 -55.315435757120504
-43.53231047942103 -55.315435757120504
-43.63231047942103 -55.315435757120504
-43.73231047942103 -55.315435757120504
-43.83231047942103 -55.315435757120504
-43.932310479421034 -55.315435757120504
-44.032310479421035 -55.315435757120504
-44.13231047942104 -55.315435757120504
-44.23231047942104 -55.315435757120504
-44.33231047942104 -55.315435757120504
-44.43231047942104 -55.315435757120504
-44.53231047942104 -55.315435757120504
-44.632

In [ ]:
from shapely.geometry import Point
def random_point_in_shp(shp):
    within = False
    while not within:
        min_lon, min_lat, max_lon, max_lat = shp.bounds
        lon = np.random.uniform(min_lon, max_lon)
        lat = np.random.uniform(min_lat, max_lat)
        within = shp.contains(Point(lon,lat))
    return Point(lon,lat)

In [ ]:
for num in range(5):
    print(random_point_in_shp(country_polygon_dict['ARG_0'][0]))

POINT (-69.52379516157526 -47.13773166111165)
POINT (-68.98693188128233 -28.37622896202876)
POINT (-68.99246823560691 -49.21493618837253)
POINT (-60.64629054176361 -37.99926344034026)
POINT (-59.05325809232092 -37.87218668805491)


In [ ]:
dict_coords

{'Latitude': [-21.83231047942072,
  -21.93231047942072,
  -22.032310479420723,
  -22.132310479420724,
  -22.232310479420725,
  -22.332310479420727,
  -22.432310479420728,
  -22.53231047942073,
  -22.63231047942073,
  -22.732310479420732,
  -22.832310479420734,
  -22.932310479420735,
  -23.032310479420737,
  -23.132310479420738,
  -23.23231047942074,
  -23.33231047942074,
  -23.432310479420742,
  -23.532310479420744,
  -23.632310479420745,
  -23.732310479420747,
  -23.832310479420748,
  -23.93231047942075,
  -24.03231047942075,
  -24.132310479420752,
  -24.232310479420754,
  -24.332310479420755,
  -24.432310479420757,
  -24.532310479420758,
  -24.63231047942076,
  -24.73231047942076,
  -24.832310479420762,
  -24.932310479420764,
  -25.032310479420765,
  -25.132310479420767,
  -25.232310479420768,
  -25.33231047942077,
  -25.43231047942077,
  -25.532310479420772,
  -25.632310479420774,
  -25.732310479420775,
  -25.832310479420777,
  -25.932310479420778,
  -26.03231047942078,
  -26.132310

In [ ]:
df = pd.DataFrame(dict_coords)
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

In [ ]:
print(gdf.head())

   POINTS  Latitude  Longitude                     geometry
0       0 -21.83231 -73.415436  POINT (-73.41544 -21.83231)
1       1 -21.93231 -73.415436  POINT (-73.41544 -21.93231)
2       2 -22.03231 -73.415436  POINT (-73.41544 -22.03231)
3       3 -22.13231 -73.415436  POINT (-73.41544 -22.13231)
4       4 -22.23231 -73.415436  POINT (-73.41544 -22.23231)


In [ ]:
point_within_arg = geopandas.sjoin(gdf, world[world.iso_a3 == 'ARG'], how="inner", op='intersects')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  """Entry point for launching an IPython kernel.


In [ ]:
point_within_arg

,POINTS,Latitude,Longitude,geometry,index_right,pop_est,continent,name,iso_a3,gdp_md_est
581,581,-49.33231,-73.315436,POINT (-73.31544 -49.33231),9,44293293,South America,Argentina,ARG,879400.0
582,582,-49.43231,-73.315436,POINT (-73.31544 -49.43231),9,44293293,South America,Argentina,ARG,879400.0
583,583,-49.53231,-73.315436,POINT (-73.31544 -49.53231),9,44293293,South America,Argentina,ARG,879400.0
584,584,-49.63231,-73.315436,POINT (-73.31544 -49.63231),9,44293293,South America,Argentina,ARG,879400.0
585,585,-49.73231,-73.315436,POINT (-73.31544 -49.73231),9,44293293,South America,Argentina,ARG,879400.0
...,...,...,...,...,...,...,...,...,...,...
60329,60329,-26.53231,-53.715436,POINT (-53.71544 -26.53231),9,44293293,South America,Argentina,ARG,879400.0
60330,60330,-26.63231,-53.715436,POINT (-53.71544 -26.63231),9,44293293,South America,Argentina,ARG,879400.0
60331,60331,-26.73231,-53.715436,POINT (-53.71544 -26.73231),9,44293293,South America,Argentina,ARG,879400.0
60332,60332,-26.83231,-53.715436,POINT (-53.71544 -26.83231),9,44293293,South America,Argentina,ARG,879400.0


In [ ]:
type(point_within_arg[point_within_arg.iso_a3 == 'ARG']) #['geometry'])

geopandas.geodataframe.GeoDataFrame

In [ ]:
# point_within_arg[point_within_arg.iso_a3 == 'ARG']['geometry']
(center_lat, center_lon) = (point_within_arg['Latitude'],	point_within_arg['Longitude'])
print((center_lat, center_lon))

(581     -49.33231
582     -49.43231
583     -49.53231
584     -49.63231
585     -49.73231
           ...   
60329   -26.53231
60330   -26.63231
60331   -26.73231
60332   -26.83231
60333   -26.93231
Name: Latitude, Length: 27566, dtype: float64, 581     -73.315436
582     -73.315436
583     -73.315436
584     -73.315436
585     -73.315436
           ...    
60329   -53.715436
60330   -53.715436
60331   -53.715436
60332   -53.715436
60333   -53.715436
Name: Longitude, Length: 27566, dtype: float64)


In [ ]:
for index, row in point_within_arg.iterrows():
  (lat, lon) = (row['Latitude'], row['Longitude'])
  # (center_lat, center_lon) = (point_within_arg['Latitude'],	point_within_arg['Longitude'])
  print(lat, lon)

Streaming output truncated to the last 5000 lines.
-37.632310479420944 -60.915435757120584
-37.732310479420946 -60.915435757120584
-37.83231047942095 -60.915435757120584
-37.93231047942095 -60.915435757120584
-38.03231047942095 -60.915435757120584
-38.13231047942095 -60.915435757120584
-38.23231047942095 -60.915435757120584
-38.332310479420954 -60.915435757120584
-38.432310479420956 -60.915435757120584
-38.53231047942096 -60.915435757120584
-38.63231047942096 -60.915435757120584
-38.73231047942096 -60.915435757120584
-38.83231047942096 -60.915435757120584
-23.93231047942075 -60.81543575712058
-24.03231047942075 -60.81543575712058
-24.132310479420752 -60.81543575712058
-24.232310479420754 -60.81543575712058
-24.332310479420755 -60.81543575712058
-24.432310479420757 -60.81543575712058
-24.532310479420758 -60.81543575712058
-24.63231047942076 -60.81543575712058
-24.73231047942076 -60.81543575712058
-24.832310479420762 -60.81543575712058
-24.932310479420764 -60.81543575712058
-25.032310479